# Análise do COVID no ES - Otimizado e Formatado
Maiky Barreto da Silva

In [ ]:
# Importando os módulos e o arquivo

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn
seaborn.set()

df = pd.read_csv('https://bi.static.es.gov.br/covid19/MICRODADOS.csv', sep = ';', encoding = 'iso8859_15')

In [ ]:
df.DataNotificacao = df.DataNotificacao.astype(np.datetime64)
df.DataObito = df.DataObito.astype(np.datetime64)

In [ ]:
df_confirmados = df.query("Classificacao == 'Confirmados'")
df_confirmados

In [ ]:
dist_confirmados = df_confirmados.groupby('DataNotificacao').count().DataCadastro

In [ ]:
rm_confirmados = df_confirmados.groupby('DataNotificacao').count().DataCadastro.rolling(14).mean()

In [ ]:
fig1 = plt.figure(figsize = (12,6))
ax1 = plt.axes()
ax1.bar(dist_confirmados[-90:-1].index, dist_confirmados[-90:-1], label = 'Mortos no dia')
ax1.plot(rm_confirmados[-90:-1], color = 'r', label = 'Média móvel de mortos nos últimos 7 dias')
ax1.legend()
ax1.set(title = 'Casos confirmados de COVID-19 nos últimos 90 dias no ES', ylabel = 'Quantidade de Casos confirmados',
        xlabel = 'Data da Notificação')

In [ ]:
df_obitos = df.query("DataObito > '2020'")
df_obitos

In [ ]:
total_obitos = df.pivot_table('DataNotificacao', 'DataObito', 'Classificacao', aggfunc='count')
total_obitos

In [ ]:
# Restringindo a data para os últimos 90d

obitos_conf = total_obitos.Confirmados[-90:-1]
obitos_susp = total_obitos.Suspeito[-90:-1]
obitos_desc = total_obitos.Descartados[-90:-1]
rm_obitos = obitos_conf.add(obitos_desc.add(obitos_susp, fill_value=0), fill_value=0).rolling(14).mean()

In [ ]:
fig2 = plt.figure(figsize=(12,6))
ax2 = plt.axes()
ax2.bar(obitos_conf.index, obitos_conf, label='Confirmados')
ax2.bar(obitos_desc.index, obitos_desc, bottom=obitos_conf, label='Descartados')
ax2.bar(obitos_susp.index, obitos_susp, bottom=obitos_conf+obitos_desc, label='Suspeitos')
ax2.plot(rm_obitos, color='r', label='Média móvel dos últimos 14d')
ax2.legend()
ax2.set(title='Composição do índice de Mortos por COVID no ES nos últimos 90 dias', xlabel='Data de Óbito',
        ylabel='Quantidade de Mortos no dia')

Filtrando os dados apenas para a GV (Serra, Cariacica, Viana, Vila Velha e Vitória)

In [ ]:
dfgv = df.query("Municipio == ['VILA VELHA','VITORIA','CARIACICA','SERRA','VIANA']")
dfgv

In [ ]:
dfgv_confirmados = dfgv.query("Classificacao == 'Confirmados'")
dfgv_confirmados

In [ ]:
casos_gv = dfgv_confirmados.pivot_table('Classificacao','DataNotificacao',aggfunc='count').Classificacao
rm_casos_gv = casos_gv.rolling(14).mean()

In [ ]:
fig3 = plt.figure(figsize=(12,6))
ax3 = plt.axes()
ax3.bar(casos_gv[-90:-1].index, casos_gv[-90:-1],label='Confirmados')
ax3.plot(rm_casos_gv[-90:-1],color='r',label='Média Móvel 14d')
ax3.legend()
ax3.set(title='Casos confirmados de COVID na GV nos últimos 90d', xlabel='Data da Notificação',
        ylabel='Pacientes notificados')

In [ ]:
total_obitos_gv = dfgv.query("DataObito > '2020'").pivot_table('DataNotificacao','DataObito','Classificacao',aggfunc='count')
total_obitos_gv

In [ ]:
# Restringindo a data para os últimos 90d

obitos_conf_gv = total_obitos_gv.Confirmados[-90:-1]
obitos_susp_gv = total_obitos_gv.Suspeito[-90:-1]
obitos_desc_gv = total_obitos_gv.Descartados[-90:-1]
rm_obitos_gv = obitos_conf_gv.add(obitos_desc_gv.add(obitos_susp_gv, fill_value=0), fill_value=0).rolling(14).mean()

In [ ]:
fig4 = plt.figure(figsize=(12,6))
ax4 = plt.axes()
ax4.bar(obitos_conf_gv.index, obitos_conf_gv, label='Confirmados')
ax4.bar(obitos_desc_gv.index, obitos_desc_gv, bottom=obitos_conf_gv, label='Descartados')
ax4.bar(obitos_susp_gv.index, obitos_susp_gv, bottom=obitos_conf_gv+obitos_desc_gv, label='Suspeitos')
ax4.plot(rm_obitos_gv, color='r', label='Média móvel dos últimos 14d')
ax4.legend()
ax4.set(title='Composição do índice de Mortos por COVID na GV nos últimos 90 dias', xlabel='Data de Óbito',
        ylabel='Quantidade de Mortos no dia')

In [ ]:
# Retirando os descartados do plot e da média móvel

rm2_obitos_gv = obitos_conf_gv.add(obitos_susp_gv, fill_value=0).rolling(14).mean()
rm3_obitos_gv = obitos_desc_gv.fillna(0).rolling(14).mean()

In [ ]:
fig5 = plt.figure(figsize=(12,6))
ax5 = plt.axes()
ax5.bar(obitos_conf_gv.index, obitos_conf_gv, label='Confirmados')
ax5.bar(obitos_desc_gv.index, obitos_desc_gv, label='Descartados')
ax5.bar(obitos_susp_gv.index, obitos_susp_gv, bottom=obitos_conf_gv, label='Suspeitos')
ax5.plot(rm3_obitos_gv, color='magenta', label='Média móvel Descartados 14d')
ax5.plot(rm2_obitos_gv, color='r', label='Média móvel Conf+Susp 14d')
ax5.legend()
ax5.set(title='Composição do índice de Mortos por COVID na GV nos últimos 90 dias', xlabel='Data de Óbito',
        ylabel='Quantidade de Mortos no dia')